### Appliquons l'approche de Julia sur les données metada

In [5]:
import pandas as pd 
metadata = pd.read_csv("\S2\TAL\Projet\data\metadata.csv")

metadata.shape

(51078, 18)

In [6]:
metadata.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
2,le0ogx1s,NaN,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...


In [3]:
# Garder uniquement les colonnes utiles
docs = metadata[["cord_uid", "title", "abstract"]].dropna(subset=["abstract"])

In [9]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Vérifier les colonnes disponibles (optionnel)
print(metadata.columns)

# Sélectionner les colonnes importantes : "title", "abstract" et "text"
# (Assurez-vous que ces colonnes existent bien dans votre fichier)
# Ici, on remplace les valeurs manquantes par une chaîne vide ("") pour éviter des erreurs
metadata["title"] = metadata["title"].fillna("")
metadata["abstract"] = metadata["abstract"].fillna("")

# Créer une colonne 'full_text' en joignant les colonnes "title", "abstract" et "text"
metadata["full_text"] = metadata["title"] + " " + metadata["abstract"]

# Initialiser le tokenizer et les stop words
tokenizer = RegexpTokenizer(r'\w+')
stop_words = ENGLISH_STOP_WORDS

# Définir la fonction de prétraitement
def preprocess(text):
    tokens = tokenizer.tokenize(text.lower())
    return " ".join([t for t in tokens if t not in stop_words])

# Appliquer le prétraitement sur la colonne full_text
metadata["processed_full_text"] = metadata["full_text"].apply(preprocess)

# Afficher un aperçu du résultat
print(metadata[["cord_uid", "title", "abstract", "processed_full_text"]].head())


Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal',
       'Microsoft Academic Paper ID', 'WHO #Covidence', 'has_pdf_parse',
       'has_pmc_xml_parse', 'full_text_file', 'url', 'full_text',
       'processed_full_text'],
      dtype='object')
   cord_uid                                              title  \
0  xqhn0vbp  Airborne rhinovirus detection and effect of ul...   
1  gi6uaa83    Discovering human history from stomach bacteria   
2  le0ogx1s     A new recruit for the army of the men of death   
3  fy4w7xz8  Association of HLA class I with severe acute r...   
4  0qaoam29   A double epidemic model for the SARS propagation   

                                            abstract  \
0  BACKGROUND: Rhinovirus, the most common cause ...   
1  Recent analyses of human pathogens have reveal...   
2  The army of the men of death, in John Bunyan's...   
3  BACKGROUND: The human leukocyte antigen 

In [11]:
documents = metadata[["cord_uid", "title", "abstract", "processed_full_text"]]

In [16]:
corpus_df = documents.dropna(subset=["processed_full_text"])

documents = corpus_df["processed_full_text"].tolist()
doc_ids = corpus_df["cord_uid"].tolist()

In [17]:
## Représentation des documents en vecteur de poids TF-IDF 

from sklearn.feature_extraction.text import TfidfVectorizer

# Initialiser le vecteur TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)  # sparse matrix


In [18]:
from sklearn.metrics.pairwise import cosine_similarity

# Exemple de requête : query_id = 1
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

tokenizer = RegexpTokenizer(r'\w+')
stop_words = ENGLISH_STOP_WORDS

def preprocess(text):
    tokens = tokenizer.tokenize(text.lower())
    return " ".join([word for word in tokens if word not in stop_words])

# Exemple d'une requête (tu pourras automatiser ça ensuite)
query = "coronavirus origin"
processed_query = preprocess(query)

# Vectoriser la requête
query_vec = vectorizer.transform([processed_query])

# Calculer la similarité cosinus
cos_sim = cosine_similarity(query_vec, tfidf_matrix).flatten()

# Récupérer les top-k documents les plus similaires
top_k = 10
top_indices = cos_sim.argsort()[::-1][:top_k]

# Affichage des résultats
for i in top_indices:
    print(f"\nScore: {cos_sim[i]:.4f}")
    print(f"Doc ID: doc_ids{[i]}")
    print(corpus_df.iloc[i]['title'])



Score: 0.6458
Doc ID: doc_ids[np.int64(39858)]
Bat origin of a new human coronavirus: there and back again

Score: 0.4850
Doc ID: doc_ids[np.int64(49243)]
Diversity of Coronaviruses in Bats: Insights Into Origin of SARS Coronavirus

Score: 0.4542
Doc ID: doc_ids[np.int64(25001)]
Coronavirus

Score: 0.3915
Doc ID: doc_ids[np.int64(30010)]
Strategies to trace back the origin of COVID-19

Score: 0.3599
Doc ID: doc_ids[np.int64(20674)]
Mosaic Evolution of the Severe Acute Respiratory Syndrome Coronavirus

Score: 0.3486
Doc ID: doc_ids[np.int64(39978)]
Bat-Origin Coronaviruses Expand Their Host Range to Pigs

Score: 0.3371
Doc ID: doc_ids[np.int64(13844)]
The prevalence, origin, and prevention of six human coronaviruses

Score: 0.3348
Doc ID: doc_ids[np.int64(42927)]
Reacting to the emergence of swine-origin influenza A H1N1

Score: 0.3270
Doc ID: doc_ids[np.int64(14467)]
New coronavirus

Score: 0.3127
Doc ID: doc_ids[np.int64(47076)]
What next for the coronavirus response?


In [19]:
import numpy as np

def precision_at_k(predicted, relevant, k):
    if not predicted or not relevant:
        return 0.0
    predicted_k = predicted[:k]
    hits = len(set(predicted_k) & set(relevant))
    return hits / k

def recall_at_k(predicted, relevant, k):
    if not predicted or not relevant:
        return 0.0
    predicted_k = predicted[:k]
    return len(set(predicted_k) & set(relevant)) / len(relevant)

def reciprocal_rank(predicted, relevant):
    for idx, doc_id in enumerate(predicted):
        if doc_id in relevant:
            return 1 / (idx + 1)
    return 0.0

def dcg_at_k(relevances, k):
    relevances = np.array(relevances)[:k]
    return np.sum(relevances / np.log2(np.arange(2, relevances.size + 2)))

def ndcg_at_k(predicted, relevant, k):
    # Relevance vector : 1 si doc dans relevant, sinon 0
    relevances = [1 if doc in relevant else 0 for doc in predicted[:k]]
    dcg = dcg_at_k(relevances, k)
    ideal_relevances = sorted(relevances, reverse=True)
    idcg = dcg_at_k(ideal_relevances, k)
    return dcg / idcg if idcg > 0 else 0.0


def evaluate_model(search_function, queries_df, qrels_df, k=10):
    precision_scores = []
    recall_scores = []
    rr_scores = []
    ndcg_scores = []

    for _, row in queries_df.iterrows():
        qid = str(row["query_id"])
        query_text = row["query"]

        relevant_docs = qrels_df[qrels_df["query_id"] == qid]["doc_id"].tolist()
        if not relevant_docs:
            continue

        predicted_docs = search_function(query_text, top_k=k)
        if not predicted_docs:
            continue

        precision_scores.append(precision_at_k(predicted_docs, relevant_docs, k))
        recall_scores.append(recall_at_k(predicted_docs, relevant_docs, k))
        rr_scores.append(reciprocal_rank(predicted_docs, relevant_docs))
        ndcg_scores.append(ndcg_at_k(predicted_docs, relevant_docs, k))

    return {
        "precision@k": np.mean(precision_scores),
        "recall@k": np.mean(recall_scores),
        "MRR": np.mean(rr_scores),
        "nDCG@k": np.mean(ndcg_scores)
    }


#### Chargement des qrels et queries pour faire les évaluations 

In [20]:
# On peut maintenant concevoir le premier modèle pour la recherche d'informations dans les données médicales que nous
# venons de charger 

## Nous avons déjà les requêtes ainsi que les bons documents (qrels) pour l'évaluation du modèle 
queries_df = pd.read_csv("\S2\TAL\Projet\data\queries_rnd1.csv")
queries_df.head()


,query_id,query,question,narrative
0,1,coronavirus origin,what is the origin of COVID-19,seeking range of information about the SARS-Co...
1,2,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the SARS-Co...
2,3,coronavirus immunity,will SARS-CoV2 infected people develop immunit...,seeking studies of immunity developed due to i...
3,4,how do people die from the coronavirus,what causes death from Covid-19?,Studies looking at mechanisms of death from Co...
4,5,animal models of COVID-19,what drugs have been active against SARS-CoV o...,Papers that describe the results of testing d...


In [21]:
qrels_df = pd.read_csv("\S2\TAL\Projet\data\qrels_rnd1.csv")
qrels_df.head()

,query_id,doc_id,relevance
0,0,010vptx3,2
1,1,02f0opkr,1
2,1,04ftw7k9,0
3,1,05qglt1f,0
4,1,0604jed8,0


In [22]:
# Forcer les query_id en str dans les deux DataFrames
qrels_df["query_id"] = qrels_df["query_id"].astype(str)
queries_df["query_id"] = queries_df["query_id"].astype(str)

# Idem pour doc_id (pour être sûr qu’il n’y a pas de soucis de comparaison)
qrels_df["doc_id"] = qrels_df["doc_id"].astype(str)


In [23]:
def search_tfidf(query_text, top_k=10):
    processed = preprocess(query_text)
    query_vec = vectorizer.transform([processed])
    scores = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = scores.argsort()[::-1][:top_k]
    return [doc_ids[i] for i in top_indices]


In [24]:
results = evaluate_model(search_tfidf, queries_df, qrels_df, k=10)

print(f"Résultats pour TF-IDF :")
for metric, score in results.items():
    print(f"- {metric}: {score:.4f}")


Résultats pour TF-IDF :
- precision@k: 0.4000
- recall@k: 0.0007
- MRR: 1.0000
- nDCG@k: 0.8224


#### Les résultats sont les mêmes donc c'est le même dataset 